In [1]:
from pathlib import Path
import pyrealsense2 as rs
import numpy as np
from tqdm import tqdm

rosbag_dir = Path("/rl_benchmark/real_robot/rosbag_recordings")
rosbag_path = rosbag_dir / "20230512_125925_test_camera_pose.bag"

# rs.log_to_console(rs.log_severity.debug)

# Create pipeline and config from rosbag
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(str(rosbag_path), repeat_playback=False)

# Start streaming from file
profile = pipeline.start(config)
playback = profile.get_device().as_playback()
playback.set_real_time(False)

# Get camera intrinsics
depth_profile = profile.get_stream(rs.stream.depth).as_video_stream_profile()
color_profile = profile.get_stream(rs.stream.color).as_video_stream_profile()
color_intrinsics = color_profile.intrinsics
width, height = color_intrinsics.width, color_intrinsics.height
fx, fy = color_intrinsics.fx, color_intrinsics.fy
cx, cy = color_intrinsics.ppx, color_intrinsics.ppy

# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align = rs.align(rs.stream.color)

# Store the frames
frames_dict = {
    "intrinsics": np.array([fx, fy, cx, cy]),
    "rgb_image": [],
    "depth_image": [],
}

with tqdm() as pbar:
    while True:
        # Get time-synchronized frames of each enabled stream in the pipeline
        frames_exist, frames = pipeline.try_wait_for_frames()
        if not frames_exist:
            break

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)
        # Verify intrinsics
        aligned_intrinsics = aligned_frames.get_profile().as_video_stream_profile().intrinsics
        np.testing.assert_allclose(frames_dict["intrinsics"],
                                   [aligned_intrinsics.fx, aligned_intrinsics.fy,
                                    aligned_intrinsics.ppx, aligned_intrinsics.ppy])

        # Get aligned frames
        depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()
        # Use copy so frame resources can be released
        depth_image = np.asanyarray(depth_frame.get_data()).copy()
        color_image = np.asanyarray(color_frame.get_data()).copy()
        
        frames_dict["rgb_image"].append(color_image)
        frames_dict["depth_image"].append(depth_image)

        pbar.update(1)

for k, v in frames_dict.items():
    frames_dict[k] = np.stack(v)
np.savez_compressed('test.npz', **frames_dict)

102it [00:05, 18.36it/s]


In [36]:
frames_dict["rgb_image"].dtype, frames_dict["depth_image"].dtype

(dtype('uint8'), dtype('uint16'))

In [10]:
import numpy as np
frames_dict = np.load("/rl_benchmark/real_robot/rosbag_recordings/20230512_125925_test_camera_pose.npz")
rgb_image = frames_dict["rgb_image"][0]
depth_image = frames_dict["depth_image"][0]

height, width = depth_image.shape

uu, vv = np.meshgrid(np.arange(width), np.arange(height))

z = depth_image / 1000.0
x = (uu - cx) * z / fx
y = (vv - cy) * z / fy
xyz_image = np.stack([x, y, z], axis=-1)
xyz_image.shape

(480, 640, 3)

In [11]:
T = np.load("Tb_b2c.npy")

xyz_image = xyz_image.reshape(-1, 3) @ T[:3, :3].T + T[:3, 3]
xyz_image = xyz_image.reshape(height, width, 3)
xyz_image.shape

(480, 640, 3)

In [12]:
from grounded_sam_track import GroundedSAMTrack
grounded_sam_track = GroundedSAMTrack(device="cuda:1")
ret = grounded_sam_track.predict_and_track_batch([rgb_image], [0], ["red cube", "green bowl"])
ret["pred_masks"][0].shape

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: '/rl_benchmark/grounded-sam/models/groundingdino_swinb_cogcoor.pth'

In [10]:
pipeline.get_active_profile().get_device().as_playback().is_real_time()

True

In [7]:
pipeline.get_active_profile().get_streams()

[<pyrealsense2.[video_]stream_profile: Depth(0) 848x480 @ 30fps Z16>,
 <pyrealsense2.[video_]stream_profile: Color(0) 640x480 @ 30fps RGB8>]

In [5]:
from pyrl.utils.lib3d.o3d_utils import np2pcd
pcd = np2pcd(xyz_image[mask], color_image[mask] / 255.0)
import open3d as o3d
coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame()
o3d.visualization.draw_geometries([pcd, coord_frame])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
import pyvista as pv
plotter = pv.Plotter(notebook=True)
plotter.add_points(xyz_image[mask], point_size=1.0,
                   scalars=color_image[mask] / 255.0, rgb=True)
plotter.add_axes()
plotter.add_bounding_box()
plotter.show(jupyter_backend='trame')

Widget(value="<iframe src='http://localhost:46295/index.html?ui=P_0x7fc2886c0fd0_1&reconnect=auto' style='widt…

In [5]:
# Get stream profile and camera intrinsics
profile = pipeline.get_active_profile()
depth_profile = rs.video_stream_profile(profile.get_stream(rs.stream.depth))
depth_intrinsics = depth_profile.get_intrinsics()
w, h = depth_intrinsics.width, depth_intrinsics.height
depth_intrinsics, w, h

([ 848x480  p[428.431 234.124]  f[428.159 428.159]  Brown Conrady [0 0 0 0 0] ],
 848,
 480)

In [6]:
# Get stream profile and camera intrinsics
profile = pipeline.get_active_profile()
color_profile = rs.video_stream_profile(profile.get_stream(rs.stream.color))
color_intrinsics = color_profile.get_intrinsics()
w, h = color_intrinsics.width, color_intrinsics.height
color_intrinsics, w, h

([ 640x480  p[324.315 241.952]  f[601.441 601.377]  Inverse Brown Conrady [0 0 0 0 0] ],
 640,
 480)